# Matrix Factorization Model End-to-End Demo

This notebook demonstrates KMR's MatrixFactorizationModel for collaborative filtering, including:

- Data generation using KMR utilities
- Model creation and training
- Recommendation generation and evaluation
- Visualization of recommendations and similarities


In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.optimizers import Adam

from kmr.models import MatrixFactorizationModel
from kmr.metrics import AccuracyAtK, PrecisionAtK, RecallAtK
from kmr.losses import ImprovedMarginRankingLoss
from kmr.utils import KMRDataGenerator, KMRPlotter

print("✅ All imports successful!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")


✅ All imports successful!
TensorFlow version: 2.18.0
Keras version: 3.8.0


## 1. Generate Collaborative Filtering Data

We'll use KMR's data generator to create synthetic user-item interaction data.


In [2]:
print("📦 Generating collaborative filtering data...")

user_ids, item_ids, ratings, user_features, item_features = KMRDataGenerator.generate_collaborative_filtering_data(
    n_users=1000,
    n_items=500,
    n_interactions=10000,
    random_state=42,
    rating_scale=(1, 5),
    sparsity=0.95
)

n_users = len(np.unique(user_ids))
n_items = len(np.unique(item_ids))

print(f"✅ Generated data:")
print(f"   - Users: {n_users}")
print(f"   - Items: {n_items}")
print(f"   - Interactions: {len(user_ids)}")
print(f"   - Rating range: {ratings.min():.1f} - {ratings.max():.1f}")
print(f"   - Average rating: {ratings.mean():.2f}")

# Convert to binary interaction (for implicit feedback)
interactions = (ratings >= 3.0).astype(np.float32)

# Split into train/test
train_size = int(0.8 * len(user_ids))
train_user_ids = tf.constant(user_ids[:train_size])
train_item_ids = tf.constant(item_ids[:train_size])
train_interactions = tf.constant(interactions[:train_size])

test_user_ids = tf.constant(user_ids[train_size:])
test_item_ids = tf.constant(item_ids[train_size:])
test_interactions = tf.constant(interactions[train_size:])


📦 Generating collaborative filtering data...
✅ Generated data:
   - Users: 1000
   - Items: 500
   - Interactions: 10000
   - Rating range: 1.0 - 5.0
   - Average rating: 2.99


## 2. Build Matrix Factorization Model


In [3]:
# Create model
model = MatrixFactorizationModel(
    num_users=n_users,
    num_items=n_items,
    embedding_dim=64,
    top_k=10,
    l2_reg=0.01
)

# Create recommendation metrics
acc_at_5 = AccuracyAtK(k=5, name="acc@5")
acc_at_10 = AccuracyAtK(k=10, name="acc@10")
prec_at_5 = PrecisionAtK(k=5, name="prec@5")
prec_at_10 = PrecisionAtK(k=10, name="prec@10")
recall_at_5 = RecallAtK(k=5, name="recall@5")
recall_at_10 = RecallAtK(k=10, name="recall@10")

# Compile model with custom ranking loss and metrics
# Model returns tuple: (similarities, rec_indices, rec_scores)
# Use list mapping: first element has loss/metrics, others are None
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=[
        ImprovedMarginRankingLoss(margin=1.0, max_min_weight=0.6, avg_weight=0.4),  # For similarities
        None,  # For rec_indices
        None   # For rec_scores
    ],
    metrics=[
        [acc_at_5, acc_at_10, prec_at_5, prec_at_10, recall_at_5, recall_at_10],  # For similarities
        None,  # For rec_indices
        None   # For rec_scores
    ]
)

print("✅ Model created and compiled!")
print(f"   - Embedding dimension: {model.embedding_dim}")
print(f"   - Top-K: {model.top_k}")
print(f"   - Metrics: Accuracy@5, Accuracy@10, Precision@5, Precision@10, Recall@5, Recall@10")

2025-11-07 13:10:56.039 | DEBUG    | kmr.layers._base_layer:_log_initialization:73 - Initialized CollaborativeUserItemEmbedding with parameters: {'name': 'collaborative_user_item_embedding', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'num_users': 1000, 'num_items': 500, 'embedding_dim': 64, 'l2_reg': 0.01}
2025-11-07 13:10:56.040 | DEBUG    | kmr.layers._base_layer:_log_initialization:73 - Initialized NormalizedDotProductSimilarity with parameters: {'name': 'normalized_dot_product_similarity', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}}
2025-11-07 13:10:56.041 | DEBUG    | kmr.layers._base_layer:_log_initialization:73 - Initialized TopKRecommendationSelector with parameters: {'name': 'top_k_recommendation_selector', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name

✅ Model created and compiled!
   - Embedding dimension: 64
   - Top-K: 10
   - Metrics: Accuracy@5, Accuracy@10, Precision@5, Precision@10, Recall@5, Recall@10


## 3. Train Model


In [4]:
print("🚀 Training Model")
print("=" * 60)
print("Using model.fit() with built-in ranking loss")
print("=" * 60)
print("The model's train_step() method handles ranking loss internally!")
print("Just prepare data and call model.fit() - no custom training loop needed.\n")

# Prepare data for keras.fit() format
# Group by user and create batches with all candidate items and binary labels
unique_users = np.unique(train_user_ids.numpy()[:50])  # Use subset for demo
batch_size = 8

# Create training data: for each user, provide all items and binary labels
train_x_user_ids = []
train_x_item_ids = []
train_y = []

for user_id in unique_users:
    if user_id >= n_users:  # Skip invalid user IDs
        continue
    
    user_items = train_item_ids.numpy()[train_user_ids.numpy() == user_id]
    positive_set = set(user_items[user_items < n_items])  # Filter valid items
    
    # Create label vector: 1 for positive items, 0 for others
    labels = np.zeros(n_items, dtype=np.float32)
    labels[list(positive_set)] = 1.0
    
    train_x_user_ids.append(user_id)
    train_x_item_ids.append(np.arange(n_items))
    train_y.append(labels)

train_x_user_ids = np.array(train_x_user_ids, dtype=np.int32)
train_x_item_ids = np.array(train_x_item_ids, dtype=np.int32)
train_y = np.array(train_y, dtype=np.float32)

print(f"Prepared training data: {len(train_x_user_ids)} users")
print(f"  - User IDs shape: {train_x_user_ids.shape}")
print(f"  - Item IDs shape: {train_x_item_ids.shape}")
print(f"  - Labels shape: {train_y.shape}")
print(f"  - Positive items per user: {train_y.sum(axis=1).mean():.1f} on average\n")

# Train using keras.fit() - the model handles ranking loss internally!
print("Training with model.fit()...")
history = model.fit(
    x=[train_x_user_ids, train_x_item_ids],
    y=train_y,
    epochs=15,
    batch_size=batch_size,
    verbose=1
)

print("\n✅ Training completed!")
print(f"Final loss: {history.history['loss'][-1]:.4f}")

# Display recommendation metrics
if 'acc@5' in history.history:
    print("\n📊 Recommendation Metrics:")
    print(f"   - Accuracy@5:  {history.history['acc@5'][-1]:.4f}")
    print(f"   - Accuracy@10: {history.history['acc@10'][-1]:.4f}")
    print(f"   - Precision@5:  {history.history['prec@5'][-1]:.4f}")
    print(f"   - Precision@10: {history.history['prec@10'][-1]:.4f}")
    print(f"   - Recall@5:  {history.history['recall@5'][-1]:.4f}")
    print(f"   - Recall@10: {history.history['recall@10'][-1]:.4f}")

print("\nNote: The model uses margin ranking loss internally.")
print("      Positive items are encouraged to rank higher than negative items.")
print("      Metrics track recommendation quality: Accuracy@K, Precision@K, Recall@K.")


🚀 Training Model
Using model.fit() with built-in ranking loss
The model's train_step() method handles ranking loss internally!
Just prepare data and call model.fit() - no custom training loop needed.

Prepared training data: 48 users
  - User IDs shape: (48,)
  - Item IDs shape: (48, 500)
  - Labels shape: (48, 500)
  - Positive items per user: 8.9 on average

Training with model.fit()...
Epoch 1/15


/Users/piotrlaczkowski/Library/Caches/pypoetry/virtualenvs/kmr-S1SSCx8j-py3.12/lib/python3.12/site-packages/keras/src/layers/layer.py:393: UserWarning: `build()` was called on layer 'matrix_factorization_model', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - acc@10: 0.0494 - acc@5: 0.0295 - loss: 1.4089 - prec@10: 0.0049 - prec@5: 0.0059 - recall@10: 0.0081 - recall@5: 0.0036                          
Epoch 2/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc@10: 0.7655 - acc@5: 0.5315 - loss: 1.0586 - prec@10: 0.0879 - prec@5: 0.1082 - recall@10: 0.1092 - recall@5: 0.0713
Epoch 3/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc@10: 1.0000 - acc@5: 0.9801 - loss: 0.7655 - prec@10: 0.1490 - prec@5: 0.2450 - recall@10: 0.1903 - recall@5: 0.1567
Epoch 4/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc@10: 1.0000 - acc@5: 1.0000 - loss: 0.6048 - prec@10: 0.2532 - prec@5: 0.3701 - recall@10: 0.3078 - recall@5: 0.2292
Epoch 5/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc@10: 1.0000 - acc@5: 1.0000 - loss: 0.4997 - prec@10: 0.4132 - prec@5: 0.6020 - recall@10: 0.4953 - recall@5: 0.3640
Epoch 6/15
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - acc@10: 1.0000 - acc@5: 1.0000 - loss: 0.4183 - prec@10: 0.5623 - prec@5: 0.8740 - re

## 4. Generate Recommendations and Visualize


In [5]:
# Generate recommendations for multiple users to check diversity
print("🔍 Checking recommendation diversity across users...")
n_sample_users = min(10, len(train_x_user_ids))
sample_user_indices = np.arange(n_sample_users)

# Get recommendations for all sample users
all_rec_indices = []
all_rec_scores = []

for i in range(n_sample_users):
    user_id = train_x_user_ids[sample_user_indices[i]]
    sample_user_id = tf.constant([user_id])
    sample_item_ids = tf.constant([np.arange(n_items)])
    
    # Model returns tuple: (similarities, rec_indices, rec_scores)
    similarities, rec_indices, rec_scores = model.predict([sample_user_id, sample_item_ids], verbose=0)
    
    rec_indices_np = rec_indices[0].numpy() if hasattr(rec_indices[0], 'numpy') else np.array(rec_indices[0])
    rec_scores_np = rec_scores[0].numpy() if hasattr(rec_scores[0], 'numpy') else np.array(rec_scores[0])
    
    all_rec_indices.append(rec_indices_np)
    all_rec_scores.append(rec_scores_np)

all_rec_indices = np.array(all_rec_indices)

# Check diversity
print(f"\n📊 Recommendation Diversity Analysis:")
print(f"   Checking {n_sample_users} users...")
unique_items_per_user = [len(np.unique(rec)) for rec in all_rec_indices]
shared_items = len(set(all_rec_indices[0]).intersection(*[set(rec) for rec in all_rec_indices[1:]]))
diversity_ratio = 1.0 - (shared_items / model.top_k)
print(f"   Shared items across all users: {shared_items}/{model.top_k}")
print(f"   Diversity ratio: {diversity_ratio:.2%}")
print(f"   Average unique items per user: {np.mean(unique_items_per_user):.1f}")

if shared_items == model.top_k:
    print(f"\n⚠️  WARNING: All users receive the same recommendations!")
    print(f"   This suggests the model may not be learning user-specific preferences.")
    print(f"   Try: increasing training epochs, adjusting learning rate, or checking data quality.")
else:
    print(f"\n✅ Recommendations are diverse across users - model is working correctly!")

# Visualize recommendation diversity
print("\n📊 Visualizing recommendation diversity...")
fig_diversity = KMRPlotter.plot_recommendation_diversity(
    all_rec_indices,
    user_ids=train_x_user_ids[sample_user_indices],
    title="Recommendation Diversity Across Users"
)
fig_diversity.show()

# Show detailed example for first user
print(f"\n📋 Detailed example for user {sample_user_indices[0]} (user_id={train_x_user_ids[sample_user_indices[0]]}):")
print(f"   Top-{model.top_k} recommended items: {all_rec_indices[0]}")
print(f"   Recommendation scores: {all_rec_scores[0]}")

# Visualize recommendation scores for first user
print("\n📊 Visualizing recommendation scores for sample user...")
fig_scores = KMRPlotter.plot_recommendation_scores(
    all_rec_scores[0],
    top_k=model.top_k,
    title=f"Recommendation Scores for User {train_x_user_ids[sample_user_indices[0]]}"
)
fig_scores.show()

🔍 Checking recommendation diversity across users...

📊 Recommendation Diversity Analysis:
   Checking 10 users...
   Shared items across all users: 0/10
   Diversity ratio: 100.00%
   Average unique items per user: 10.0

✅ Recommendations are diverse across users - model is working correctly!

📊 Visualizing recommendation diversity...



📋 Detailed example for user 0 (user_id=7):
   Top-10 recommended items: [ 23 249 275 450 210 118  81 128 366  86]
   Recommendation scores: [0.94178385 0.9387672  0.93399817 0.87472117 0.86898464 0.8294649
 0.732464   0.70940304 0.45580384 0.35180512]

📊 Visualizing recommendation scores for sample user...
